# Rule merging 

---

## 1. Importing necessary modules

In [1]:
from utils.N01_agent_setup import (
    # Fact classes for our rule engine
    TelemetryFact,
    DegradationFact,
    GapFact,
    RadioFact,
    RaceStatusFact,
    StrategyRecommendation,
    F1StrategyEngine,
    
    # Utility functions for data transformation
    transform_tire_predictions,
    load_tire_predictions,
    transform_lap_time_predictions,
    load_lap_time_predictions,
    transform_radio_analysis,
    process_radio_message,
    transform_gap_data_with_consistency,
    load_gap_data,
    calculate_gap_consistency
)

# Import the rule engines from each domain
# Tire degradation rules
from utils.N02_degradation_time_rules import F1DegradationRules

# Lap time prediction rules
from utils.N03_lap_time_rules import F1LapTimeRules

# Radio communication analysis rules
from utils.N04_nlp_rules import F1RadioRules  

# Gap analysis rules
from utils.N05_gap_rules import F1GapRules

# Import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import sys
import json
import time

# Import Experta components
from experta import Rule, NOT, OR, AND, AS, MATCH, TEST, EXISTS
from experta import DefFacts, Fact, Field, KnowledgeEngine

# Add parent directory to path to access modules
sys.path.append(os.path.abspath('../'))

# Configure visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context("notebook", font_scale=1.2)

print("All components imported successfully!")

Engine initialized with 2 facts
Initial facts: [InitialFact(), RaceStatusFact(lap=1, total_laps=60, race_phase='start', track_status='clear')]

=== TIRE DEGRADATION ANALYSIS ===
Using first predicted rate as current degradation: 0.07
Tire facts declared: {'degradation': DegradationFact(degradation_rate=0.07, predicted_rates=frozenlist([0.07, 0.09, 0.12])), 'telemetry': TelemetryFact(tire_age=4, compound_id=2, driver_number=44, position=1)}
Engine now has 4 facts

=== LAP TIME PREDICTION ===
Lap time facts declared: {'telemetry': TelemetryFact(driver_number=44, lap_time=80.3, predicted_lap_time=79.9, compound_id=2, tire_age=4, position=1)}
Engine now has 5 facts

=== RADIO ANALYSIS ===
Radio fact declared: <f-5>
Engine now has 6 facts

=== ALL ENGINE FACTS ===
Fact 1: InitialFact - <f-0>
Fact 2: RaceStatusFact - <f-1>
Fact 3: DegradationFact - <f-2>
Fact 4: TelemetryFact - <f-3>
Fact 5: TelemetryFact - <f-4>
Fact 6: RadioFact - <f-5>
Successfully loaded data from ../../outputs/week5/tir

,Stint,SpeedI1,SpeedI2,SpeedFL,SpeedST,Position,LapsSincePitStop,DRSUsed,TeamID,CompoundID,TyreAge,FuelLoad,DriverNumber,FuelAdjustedLapTime,FuelAdjustedDegPercent,DegradationRate,RaceLap,PreviousRates
0,1.0,256.0,261.0,276.0,275.0,1,1.0,0,9,2,1,0.9848,1,83.935,0.000000,0.000000,1.0,[0.0]
16,1.0,252.0,257.0,276.0,295.0,1,2.0,0,9,2,2,0.9697,1,80.457,-4.143683,-3.941625,2.0,"[0.0, -3.941624999999988]"
20,1.0,249.0,256.0,276.0,297.0,1,3.0,0,9,2,3,0.9545,1,80.609,-3.962590,0.232092,3.0,"[0.0, -3.941624999999988, 0.2320916666666619]"
48,1.0,255.0,256.0,276.0,300.0,1,4.0,0,9,2,4,0.9394,1,80.511,-4.079347,0.073592,4.0,"[-3.941624999999988, 0.2320916666666619, 0.073..."
50,1.0,254.0,256.0,277.0,301.0,1,5.0,0,9,2,5,0.9242,1,80.503,-4.088878,0.007353,5.0,"[0.2320916666666619, 0.0735921568627446, 0.007..."


Successfully imported lap prediction module
Libraries and fact classes loaded successfully.
All components imported successfully!


---

## 2. Main Engine Class

This class inherits from all the enignes defined for each topic, and also a conflic resolution capability to avoid contradictory strategies being fired at the same time.

In [2]:
class F1CompleteStrategyEngine(F1DegradationRules, F1LapTimeRules, F1RadioRules, F1GapRules):
    """
    Unified strategy engine that integrates all rule systems:
    - Tire degradation rules 
    - Lap time prediction rules
    - Radio communication analysis rules
    - Gap analysis rules
    
    This class inherits from all specialized rule engines to combine their rules
    and adds conflict resolution capabilities.
    """
    
    def __init__(self):
        """Initialize the integrated engine"""
        # Call the parent constructor
        super().__init__()
        # Track which rule systems have fired rules
        self.active_systems = {
            'degradation': False,
            'lap_time': False,
            'radio': False,
            'gap': False
        }
    
    def get_recommendations(self):
        """
        Get all recommendations from the rule engine with enhanced conflict resolution.
        
        Returns:
            list: Sorted list of recommendations with conflicts resolved
        """
        # Get the base recommendations from parent method
        all_recommendations = super().get_recommendations()
        
        # If we have very few recommendations, no need for complex conflict resolution
        if len(all_recommendations) <= 2:
            return all_recommendations
        
        # Group recommendations by driver
        driver_recommendations = {}
        for rec in all_recommendations:
            driver = rec.get('DriverNumber', 0)  # Default to 0 if driver is not specified
            if driver not in driver_recommendations:
                driver_recommendations[driver] = []
            driver_recommendations[driver].append(rec)
        
        # Process each driver's recommendations for conflicts
        resolved_recommendations = []
        for driver, recs in driver_recommendations.items():
            # Only need conflict resolution if multiple recommendations
            if len(recs) > 1:
                resolved = self._resolve_conflicts(recs)
                resolved_recommendations.extend(resolved)
            else:
                # Single recommendation, no conflicts to resolve
                resolved_recommendations.extend(recs)
        
        # Sort by priority and confidence
        return sorted(
            resolved_recommendations,
            key=lambda x: (x.get('priority', 0), x.get('confidence', 0)),
            reverse=True
        )
    
    def _resolve_conflicts(self, recommendations):
        """
        Resolve conflicts between recommendations for the same driver.
        
        This method looks for contradictory recommendations and resolves them
        based on priority, confidence, and the nature of the conflict.
        
        Args:
            recommendations: List of recommendations for a single driver
            
        Returns:
            list: Resolved list of recommendations
        """
        # Group recommendations by action type
        action_groups = {}
        for rec in recommendations:
            action = rec['action']
            if action not in action_groups:
                action_groups[action] = []
            action_groups[action].append(rec)
        
        # Define conflicting action pairs
        conflicting_pairs = [
            # Can't extend stint and pit at the same time
            ('extend_stint', 'pit_stop'),
            ('extend_stint', 'prioritize_pit'),
            ('extend_stint', 'defensive_pit'),
            ('extend_stint', 'consider_pit'),
            
            # No need for preparation if immediate pit is recommended
            ('prepare_pit', 'pit_stop'),
            ('prepare_pit', 'prioritize_pit'),
            
            # Can't do undercut and overcut at the same time
            ('perform_undercut', 'perform_overcut')
        ]
        
        # Check for each conflict pair
        resolved = []
        excluded_recommendations = set()
        
        for action1, action2 in conflicting_pairs:
            if action1 in action_groups and action2 in action_groups:
                # We have a conflict!
                group1 = action_groups[action1]
                group2 = action_groups[action2]
                
                # Get the highest priority/confidence recommendation from each group
                best1 = max(group1, key=lambda x: (x.get('priority', 0), x.get('confidence', 0)))
                best2 = max(group2, key=lambda x: (x.get('priority', 0), x.get('confidence', 0)))
                
                # Compare and keep only the better one
                if (best1.get('priority', 0), best1.get('confidence', 0)) >= (best2.get('priority', 0), best2.get('confidence', 0)):
                    # best1 wins, exclude all from group2
                    excluded_recommendations.update(id(r) for r in group2)
                else:
                    # best2 wins, exclude all from group1
                    excluded_recommendations.update(id(r) for r in group1)
        
        # Add non-excluded recommendations
        for rec in recommendations:
            if id(rec) not in excluded_recommendations:
                resolved.append(rec)
        
        # Enhance the winning recommendations with context from conflicting ones
        if len(resolved) < len(recommendations):
            # We had conflicts and resolved them
            for rec in resolved:
                rec['explanation'] += " (Selected as optimal strategy after resolving conflicts)"
        
        return resolved
    
    def record_rule_fired(self, rule_name):
        """
        Record which rule fired and track which rule system it belongs to.
        
        Args:
            rule_name: Name of the rule that fired
        """
        # Standard recording from parent class
        super().record_rule_fired(rule_name)
        
        # Also track which system the rule belongs to
        if rule_name.startswith(('high_degradation', 'stint_extension', 'early_degradation')):
            self.active_systems['degradation'] = True
        elif rule_name.startswith(('optimal_performance', 'performance_cliff', 'post_traffic')):
            self.active_systems['lap_time'] = True
        elif rule_name.startswith(('grip_issue', 'weather_information', 'incident_reaction')):
            self.active_systems['radio'] = True
        elif rule_name.startswith(('undercut_opportunity', 'defensive_pit', 'strategic_overcut')):
            self.active_systems['gap'] = True

### 2.1 Testing class initialization

In [3]:
# Test that our integrated engine initializes correctly
try:
    engine = F1CompleteStrategyEngine()
    print("✅ Successfully created the integrated engine!")
    
    # Use reset to initialize the engine's working memory
    engine.reset()
    print("✅ Engine reset successful!")
    
    # Check MRO (Method Resolution Order) to confirm correct inheritance
    print("\nMethod Resolution Order:")
    for cls in F1CompleteStrategyEngine.__mro__:
        print(f"  - {cls.__name__}")
    
    # Print info about active systems
    print("\nActive rule systems:", engine.active_systems)
    
except Exception as e:
    print(f"❌ Error: {str(e)}")

✅ Successfully created the integrated engine!
✅ Engine reset successful!

Method Resolution Order:
  - F1CompleteStrategyEngine
  - F1DegradationRules
  - F1LapTimeRules
  - F1RadioRules
  - F1GapRules
  - F1StrategyEngine
  - KnowledgeEngine
  - object

Active rule systems: {'degradation': False, 'lap_time': False, 'radio': False, 'gap': False}


---

## 3. Transformation Function for integrating all the facts

### `prepare_integrated_facts`: Transforming data into engine facts

This function serves as the crucial bridge that converts various data types into facts that our rule engine can process. It handles the complex task of integrating diverse sources of information into a coherent set of facts.

### Inputs:
- **Multiple data sources** (lap times, tire data, gaps, radio communications)
- **Contextual information** (driver number, current lap)

### Processing by data type:

1. **Tire degradation data**:
   - Transforms predictions into `DegradationFact` objects
   - Extracts metrics like degradation rate and future predictions
   - Captures basic telemetry information (tire age, compound)

2. **Lap time data**:
   - Processes lap time predictions by creating or updating `TelemetryFact`
   - **Key intelligence**: If telemetry information already exists, it only updates relevant fields without overwriting other data

3. **Gap data between cars**:
   - Ensures consistency calculations exist (how many laps a gap has remained similar)
   - Creates `GapFact` objects with information about cars ahead/behind and trends

4. **Radio analysis**:
   - Processes radio messages in JSON format
   - Creates `RadioFact` objects with sentiment analysis, intentions and detected entities

5. **Race status**:
   - Automatically calculates race phase (start/middle/end)
   - Creates a `RaceStatusFact` with this contextual information

### Output:
A structured dictionary with all facts organized by category, ready to be declared in the rule engine.

In [4]:
def prepare_integrated_facts(
    lap_data=None,            # Lap time data
    tire_predictions=None,    # Tire degradation predictions
    gap_data=None,            # Gaps between cars
    radio_analysis=None,      # Radio communication analysis
    driver_number=None,       # Specific driver number
    current_lap=None,         # Current lap number
    total_laps=None           # Total race laps
):
    """
    Prepare a complete set of facts for the integrated strategy engine.
    
    This function takes various data inputs and transforms them into facts
    that can be used by our rule engine. It handles cases where some data
    might not be available.
    
    Args:
        lap_data: DataFrame with lap time data
        tire_predictions: DataFrame with tire degradation predictions
        gap_data: DataFrame with gap data
        radio_analysis: JSON or dict with radio analysis
        driver_number: Specific driver number to focus on
        current_lap: Current lap number
        total_laps: Total race laps
        
    Returns:
        dict: Dictionary of facts categorized by type
    """
    facts = {}
    
    print(f"Preparing facts for driver {driver_number}, lap {current_lap}")
    
    # 1. Process tire degradation data if available
    if tire_predictions is not None and driver_number is not None:
        try:
            tire_facts = transform_tire_predictions(tire_predictions, driver_number)
            if tire_facts:
                facts['degradation'] = tire_facts.get('degradation')
                # Only add telemetry if not already present
                if 'telemetry' not in facts:
                    facts['telemetry'] = tire_facts.get('telemetry')
                print(f"✅ Tire degradation facts prepared")
            else:
                print(f"⚠️ No tire facts generated for driver {driver_number}")
        except Exception as e:
            print(f"❌ Error processing tire data: {str(e)}")
    
    # 2. Process lap time data if available
    if lap_data is not None and driver_number is not None:
        try:
            lap_facts = transform_lap_time_predictions(lap_data, driver_number)
            if lap_facts:
                # Merge with existing telemetry if present, otherwise create new
                if 'telemetry' in facts and facts['telemetry'] is not None:
                    # Update existing telemetry with lap time info
                    if 'lap_time' in lap_facts['telemetry']:
                        facts['telemetry']['lap_time'] = lap_facts['telemetry']['lap_time']
                    if 'predicted_lap_time' in lap_facts['telemetry']:
                        facts['telemetry']['predicted_lap_time'] = lap_facts['telemetry']['predicted_lap_time']
                else:
                    facts['telemetry'] = lap_facts.get('telemetry')
                print(f"✅ Lap time facts prepared")
            else:
                print(f"⚠️ No lap time facts generated for driver {driver_number}")
        except Exception as e:
            print(f"❌ Error processing lap time data: {str(e)}")
    
    # 3. Process gap data if available
    if gap_data is not None and driver_number is not None:
        try:
            # First ensure gap data has consistency calculations
            if 'consistent_gap_ahead_laps' not in gap_data.columns:
                gap_data = calculate_gap_consistency(gap_data)
            
            # Transform the gap data into facts
            gap_fact = transform_gap_data_with_consistency(gap_data, driver_number)
            if gap_fact:
                facts['gap'] = gap_fact
                print(f"✅ Gap facts prepared")
            else:
                print(f"⚠️ No gap facts generated for driver {driver_number}")
        except Exception as e:
            print(f"❌ Error processing gap data: {str(e)}")
    
    # 4. Process radio analysis if available
    if radio_analysis is not None:
        try:
            # If radio_analysis is a path to a JSON file
            if isinstance(radio_analysis, str) and radio_analysis.endswith('.json'):
                radio_fact = transform_radio_analysis(radio_analysis)
            # If it's already a parsed dictionary
            elif isinstance(radio_analysis, dict):
                # Create a temporary JSON file
                import tempfile
                import json
                with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as tmp:
                    json.dump(radio_analysis, tmp)
                    tmp_path = tmp.name
                radio_fact = transform_radio_analysis(tmp_path)
                # Clean up temp file
                try:
                    os.remove(tmp_path)
                except:
                    pass
            
            if radio_fact:
                facts['radio'] = radio_fact
                print(f"✅ Radio analysis facts prepared")
            else:
                print(f"⚠️ No radio facts generated")
        except Exception as e:
            print(f"❌ Error processing radio data: {str(e)}")
    
    # 5. Create race status fact if lap info is available
    if current_lap is not None and total_laps is not None:
        try:
            from utils.N01_agent_setup import calculate_race_phase
            race_phase = calculate_race_phase(current_lap, total_laps)
            facts['race_status'] = RaceStatusFact(
                lap=int(current_lap),
                total_laps=int(total_laps),
                race_phase=race_phase,
                track_status="clear"  # Default to clear if no info
            )
            print(f"✅ Race status fact prepared (lap {current_lap}, phase: {race_phase})")
        except Exception as e:
            print(f"❌ Error creating race status fact: {str(e)}")
    
    # Summary
    fact_types = [k for k, v in facts.items() if v is not None]
    print(f"Prepared {len(fact_types)} fact types: {fact_types}")
    
    return facts

---

## 4. Load and prepare main function
### `load_all_race_data`: Loading and preparing data from sources

This function handles the entire initial process of obtaining and preparing data before transformation:

### Inputs:
- **Paths to different sources** of data (CSV files, models, etc.)

### Process:

1. **Loading lap time data**:
   - Reads the main CSV with lap information
   - **Advanced functionality**: If model path is provided, automatically generates:
     - Tire degradation predictions
     - Future lap time predictions

2. **Loading gap data**:
   - Reads the CSV with analysis of gaps between cars
   - Automates the calculation of consistency metrics (how long a gap remains stable)

3. **Loading radio analysis**:
   - Handles both directories (multiple messages) and individual files
   - Reads and structures NLP analyses of communications

### Special features:
- **Robust error handling**: Each component has its own try-except block
- **Automatic prediction generation**: Uses existing ML models to enrich basic data
- **Detailed feedback**: Provides clear information about which data was loaded and which failed

### Output:
A complete dictionary with all loaded and preprocessed data, ready for transformation into facts.

The combination of these two functions fully automates the flow from raw data to inference-ready facts, allowing the integrated engine to generate recommendations based on multiple sources of information.

In [5]:
def load_all_race_data(
    lap_data_path=None,
    gap_data_path=None,
    radio_path=None,
    models_path=None
):
    """
    Load and prepare all race data from various sources.
    
    This function handles loading various data types needed for
    strategy analysis from files or other sources.
    
    Args:
        lap_data_path: Path to CSV with lap and telemetry data
        gap_data_path: Path to CSV with processed gap data
        radio_path: Path to directory with radio analysis JSONs
        models_path: Path to prediction models directory
        
    Returns:
        dict: Dictionary containing all loaded data
    """
    data = {}
    
    # 1. Load lap time data if available
    if lap_data_path and os.path.exists(lap_data_path):
        try:
            lap_data = pd.read_csv(lap_data_path)
            data['lap_data'] = lap_data
            print(f"✅ Loaded lap data: {len(lap_data)} rows")
            
            # If models path is provided, generate tire predictions
            if models_path and os.path.exists(models_path):
                try:
                    tire_predictions = load_tire_predictions(lap_data, models_path)
                    if tire_predictions is not None:
                        data['tire_predictions'] = tire_predictions
                        print(f"✅ Generated tire predictions: {len(tire_predictions)} rows")
                except Exception as e:
                    print(f"❌ Error generating tire predictions: {str(e)}")
            
            # Also try to load lap time predictions
            try:
                lap_predictions = load_lap_time_predictions(lap_data, models_path)
                if lap_predictions is not None:
                    data['lap_predictions'] = lap_predictions
                    print(f"✅ Generated lap time predictions: {len(lap_predictions)} rows")
            except Exception as e:
                print(f"❌ Error generating lap time predictions: {str(e)}")
                
        except Exception as e:
            print(f"❌ Error loading lap data: {str(e)}")
    
    # 2. Load gap data if available
    if gap_data_path and os.path.exists(gap_data_path):
        try:
            gap_data = pd.read_csv(gap_data_path)
            # Ensure consistency metrics are calculated
            if 'consistent_gap_ahead_laps' not in gap_data.columns:
                gap_data = calculate_gap_consistency(gap_data)
            data['gap_data'] = gap_data
            print(f"✅ Loaded gap data: {len(gap_data)} rows")
        except Exception as e:
            print(f"❌ Error loading gap data: {str(e)}")
    
    # 3. Load radio analysis if available
    if radio_path and os.path.exists(radio_path):
        # Check if it's a directory or a single file
        if os.path.isdir(radio_path):
            radio_files = [f for f in os.listdir(radio_path) if f.endswith('.json')]
            data['radio_analyses'] = []
            for file in radio_files:
                file_path = os.path.join(radio_path, file)
                try:
                    with open(file_path, 'r') as f:
                        radio_analysis = json.load(f)
                    data['radio_analyses'].append({
                        'path': file_path,
                        'data': radio_analysis
                    })
                except Exception as e:
                    print(f"❌ Error loading radio file {file}: {str(e)}")
            print(f"✅ Loaded {len(data['radio_analyses'])} radio analyses")
        elif radio_path.endswith('.json'):
            try:
                with open(radio_path, 'r') as f:
                    radio_analysis = json.load(f)
                data['radio_analyses'] = [{
                    'path': radio_path,
                    'data': radio_analysis
                }]
                print(f"✅ Loaded single radio analysis")
            except Exception as e:
                print(f"❌ Error loading radio file: {str(e)}")
    
    # Summary
    print(f"\nLoaded data summary:")
    for key, value in data.items():
        if isinstance(value, list):
            print(f"- {key}: {len(value)} items")
        elif isinstance(value, pd.DataFrame):
            print(f"- {key}: {len(value)} rows, {len(value.columns)} columns")
        else:
            print(f"- {key}: {type(value)}")
    
    return data

---

In [6]:
from utils.N03_lap_time_rules import test_with_realistic_approach

## 5. Complete end-to-end pipeline

In [7]:
def generate_integrated_strategy_recommendations(
    race_data,
    driver_numbers=None,
    specific_lap=None,
    models_path=None,
    enable_radio=True,
    enable_gaps=True
):
    """
    Complete end-to-end pipeline for generating strategic recommendations using
    the integrated rule engine with all data sources.
    
    This function handles the entire process:
    1. Preparing facts from race data
    2. Initializing the integrated engine
    3. Generating recommendations for each driver
    
    Args:
        race_data: Dictionary with all race data (lap_data, tire_predictions, gap_data, radio_analyses)
        driver_numbers: List of driver numbers to analyze (None = analyze all)
        specific_lap: Specific lap to analyze (None = use latest lap per driver)
        models_path: Path to prediction models directory
        enable_radio: Whether to use radio communication analysis
        enable_gaps: Whether to use gap analysis
        
    Returns:
        DataFrame with all recommendations from all analyzed drivers
    """
    # Track execution time
    start_time = time.time()
    print(f"Starting integrated strategy analysis at {datetime.now().strftime('%H:%M:%S')}")
    
    # Extract components from race_data
    lap_data = race_data.get('lap_data')
    tire_predictions = race_data.get('tire_predictions')
    gap_data = race_data.get('gap_data')
    radio_analyses = race_data.get('radio_analyses', [])
     # Check and fix missing LapNumber column
    if 'LapNumber' not in lap_data.columns:
        print("Creating LapNumber column based on Stint and TyreAge...")
        
        # Check if we have the necessary columns
        if 'Stint' in lap_data.columns and 'TyreAge' in lap_data.columns:
            # Calculate the maximum TyreAge for each stint to get stint lengths
            max_age_by_stint = lap_data.groupby(['DriverNumber', 'Stint'])['TyreAge'].max().reset_index()
            max_age_by_stint = max_age_by_stint.rename(columns={'TyreAge': 'StintLength'})
            
            # Create cumulative stint lengths for each driver
            stint_lengths = {}
            for driver in lap_data['DriverNumber'].unique():
                driver_stints = max_age_by_stint[max_age_by_stint['DriverNumber'] == driver]
                
                # Calculate cumulative lengths
                cumulative_lengths = [0]  # Start with 0 for the first stint
                for i in range(len(driver_stints) - 1):
                    cumulative_lengths.append(
                        cumulative_lengths[-1] + driver_stints.iloc[i]['StintLength']
                    )
                
                # Store in dictionary
                stint_lengths[driver] = {
                    stint: length for stint, length in zip(
                        driver_stints['Stint'], cumulative_lengths
                    )
                }
            
            # Function to calculate race lap
            def calculate_lap_number(row):
                driver = row['DriverNumber']
                stint = row['Stint']
                tyre_age = row['TyreAge']
                
                # Get the starting lap for this stint
                start_lap = stint_lengths.get(driver, {}).get(stint, 0)
                
                # Add current TyreAge to get race lap
                return start_lap + tyre_age
            
            # Apply function to calculate LapNumber
            lap_data['LapNumber'] = lap_data.apply(calculate_lap_number, axis=1)
        
        elif 'TyreAge' in lap_data.columns:
            # Simplified approach if we only have TyreAge
            print("Using TyreAge as LapNumber (assuming single stint)")
            lap_data['LapNumber'] = lap_data['TyreAge']
        
        else:
            # Fallback: create sequential numbers per driver
            print("Creating sequential lap numbers per driver")
            lap_data['LapNumber'] = lap_data.groupby('DriverNumber').cumcount() + 1
    # Ensure we have the basic required data
    if lap_data is None:
        raise ValueError("Lap data is required for strategy analysis")
    
    # Get total race laps
    total_laps = lap_data['LapNumber'].max() if 'LapNumber' in lap_data.columns else 50
    print(f"Total race laps detected: {total_laps}")
    
    # Determine drivers to analyze
    if driver_numbers is None:
        driver_numbers = lap_data['DriverNumber'].unique()
    
    print(f"Will analyze {len(driver_numbers)} drivers: {driver_numbers}")
    
    # Initialize container for all recommendations
    all_recommendations = []
    
    # Process each driver
    for driver_number in driver_numbers:
        driver_start_time = time.time()
        print(f"\n{'='*50}")
        print(f"ANALYZING DRIVER {driver_number}")
        print(f"{'='*50}")
        
        # Filter lap data for this driver
        driver_lap_data = lap_data[lap_data['DriverNumber'] == driver_number]
        if driver_lap_data.empty:
            print(f"No lap data for driver {driver_number}, skipping...")
            continue
        
        # Determine which lap to analyze
        if specific_lap is not None:
            # Use the specified lap if available
            lap_to_analyze = specific_lap
            if lap_to_analyze not in driver_lap_data['LapNumber'].values:
                print(f"Specified lap {lap_to_analyze} not found for driver {driver_number}")
                closest_lap = driver_lap_data['LapNumber'].astype(int).values
                closest_lap = closest_lap[np.abs(closest_lap - lap_to_analyze).argmin()]
                print(f"Using closest available lap: {closest_lap}")
                lap_to_analyze = closest_lap
        else:
            # Use the latest lap available
            lap_to_analyze = driver_lap_data['LapNumber'].max()
        
        print(f"Analyzing lap {lap_to_analyze} for driver {driver_number}")
        
        # Find position for this driver at this lap
        try:
            driver_position = driver_lap_data[driver_lap_data['LapNumber'] == lap_to_analyze]['Position'].iloc[0]
            print(f"Driver position: P{int(driver_position)}")
        except:
            driver_position = 0
            print("Could not determine driver position")
        
        # Get driver name if available
        try:
            if 'Driver' in driver_lap_data.columns:
                driver_name = driver_lap_data['Driver'].iloc[0]
            elif gap_data is not None and 'Driver' in gap_data.columns:
                driver_name = gap_data[gap_data['DriverNumber'] == driver_number]['Driver'].iloc[0]
            else:
                driver_name = f"Driver-{driver_number}"
            print(f"Driver name: {driver_name}")
        except:
            driver_name = f"Driver-{driver_number}"
        
        # Get team if available
        try:
            if 'Team' in driver_lap_data.columns:
                team = driver_lap_data['Team'].iloc[0]
            elif gap_data is not None and 'Team' in gap_data.columns:
                team = gap_data[gap_data['DriverNumber'] == driver_number]['Team'].iloc[0]
            else:
                team = "Unknown"
            print(f"Team: {team}")
        except:
            team = "Unknown"
        
        # Prepare filtered data for the specific lap we're analyzing
        lap_filtered_data = driver_lap_data[driver_lap_data['LapNumber'] == lap_to_analyze].copy()
        
        # For gaps, find the gap data for this lap
        gap_filtered_data = None
        if gap_data is not None and enable_gaps:
            gap_filtered_data = gap_data[
                (gap_data['DriverNumber'] == driver_number) & 
                (gap_data['LapNumber'] == lap_to_analyze)
            ].copy()
            if gap_filtered_data.empty:
                print(f"No gap data for driver {driver_number} at lap {lap_to_analyze}")
                gap_filtered_data = None
        
        # For radio, find the most recent radio message
        radio_filtered = None
        if radio_analyses and enable_radio:
            # This is simplified - in reality, we would need to match radio timestamps
            # with lap times to determine the most relevant radio message
            radio_filtered = radio_analyses[0]['data'] if radio_analyses else None
            if radio_filtered:
                print(f"Using radio message: '{radio_filtered.get('message', 'Unknown message')}'")
        
        # Prepare all facts for the engine
        facts = prepare_integrated_facts(
            lap_data=lap_filtered_data,
            tire_predictions=tire_predictions,
            gap_data=gap_filtered_data,
            radio_analysis=radio_filtered,
            driver_number=driver_number,
            current_lap=lap_to_analyze,
            total_laps=total_laps
        )
        
        # Check if we have enough facts to run the engine
        if len(facts) <= 1:  # Only race_status is not enough
            print(f"Insufficient facts for driver {driver_number}, skipping...")
            continue
        
        # Initialize the integrated engine
        engine = F1CompleteStrategyEngine()
        engine.reset()
        
        # Declare all facts to the engine
        print("\nDeclaring facts to the engine...")
        for fact_type, fact in facts.items():
             if fact is not None:
                print(f"Declaring {fact_type}: {fact}")
                engine.declare(fact)
                print(f"  ✓ Declared {fact_type} fact")
        
        # Run the engine
        print("\nRunning integrated strategy engine...")
        engine.run()
        
        # Get recommendations
        recommendations = engine.get_recommendations()
        print(f"\nGenerated {len(recommendations)} recommendations")
        
        # Add driver-specific metadata to recommendations
        for rec in recommendations:
            rec['DriverNumber'] = driver_number
            rec['DriverName'] = driver_name
            rec['LapNumber'] = lap_to_analyze
            rec['Position'] = driver_position
            rec['Team'] = team
            # Add which systems were active
            for system, active in engine.active_systems.items():
                rec[f'System_{system}'] = active
        
        # Add to overall results
        all_recommendations.extend(recommendations)
        
        # Print summary of which rule systems were active
        print("\nActive rule systems:")
        for system, active in engine.active_systems.items():
            status = "✓ ACTIVE" if active else "✗ INACTIVE"
            print(f"  - {system.upper()}: {status}")
        
        # Calculate and print execution time for this driver
        driver_time = time.time() - driver_start_time
        print(f"\nAnalysis for driver {driver_number} completed in {driver_time:.2f} seconds")
    
    # Process results
    if all_recommendations:
        # Convert to DataFrame
        results_df = pd.DataFrame(all_recommendations)
        
        # Sort by priority and confidence
        results_df = results_df.sort_values(
            ['DriverNumber', 'priority', 'confidence'],
            ascending=[True, False, False]
        )
        
        # Calculate execution time
        total_time = time.time() - start_time
        print(f"\nStrategy analysis completed in {total_time:.2f} seconds")
        print(f"Generated {len(results_df)} recommendations for {len(driver_numbers)} drivers")
        
        return results_df
    else:
        print("No recommendations were generated")
        return pd.DataFrame()

In [8]:
def run_complete_strategy_analysis(
    lap_data_path,
    gap_data_path=None,
    radio_path=None,
    models_path=None,
    driver_numbers=None,
    specific_lap=None,
    save_results=True,
    output_path='strategy_recommendations.csv'
):
    """
    Complete one-step function to load data and generate strategy recommendations.
    
    This is the main function that users would call directly to perform a full
    strategy analysis from data files.
    
    Args:
        lap_data_path: Path to CSV with lap and telemetry data
        gap_data_path: Path to CSV with processed gap data
        radio_path: Path to directory or file with radio analysis
        models_path: Path to prediction models directory
        driver_numbers: List of driver numbers to analyze (None = all drivers)
        specific_lap: Specific lap to analyze (None = latest lap)
        save_results: Whether to save results to CSV
        output_path: Path to save results CSV
        
    Returns:
        DataFrame with all recommendations
    """
    print(f"Starting complete strategy analysis at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Step 1: Load all race data
    print("\n1. LOADING RACE DATA")
    print("----------------------")
    race_data = load_all_race_data(
        lap_data_path=lap_data_path,
        gap_data_path=gap_data_path,
        radio_path=radio_path,
        models_path=models_path
    )
    
    # Step 2: Generate strategy recommendations
    print("\n2. GENERATING STRATEGY RECOMMENDATIONS")
    print("--------------------------------------")
    recommendations = generate_integrated_strategy_recommendations(
        race_data=race_data,
        driver_numbers=driver_numbers,
        specific_lap=specific_lap,
        models_path=models_path,
        enable_radio=(radio_path is not None),
        enable_gaps=(gap_data_path is not None)
    )
    
    # Step 3: Save results if requested
    if save_results and not recommendations.empty:
        print("\n3. SAVING RESULTS")
        print("----------------")
        try:
            recommendations.to_csv(output_path, index=False)
            print(f"Results saved to: {output_path}")
        except Exception as e:
            print(f"Error saving results: {str(e)}")
    
    return recommendations

---

In [9]:
# Test the complete pipeline with available data
try:
    # Define paths to your data files
    # Adjust these paths to match your actual file locations
    lap_data_path = '../../outputs/week3/lap_prediction_data.csv'
    gap_data_path = '../../outputs/week5/gaps_spain_2023_data.csv'  # If available
    radio_path = "../../outputs/week4/json/radio_analysis_20250417_191807.json"
    models_path = '../../outputs/week5/models/'  # Path to prediction models
    
    # Optional parameters
    driver_numbers = None  # Example: Analyze only Verstappen (#1) and Hamilton (#44)
    specific_lap = 20  # Analyze a specific lap, or None for latest
    
    # Run the complete analysis
    print(f"Testing complete strategy pipeline...")
    results = run_complete_strategy_analysis(
        lap_data_path=lap_data_path,
        gap_data_path=gap_data_path,
        radio_path=radio_path,
        models_path=models_path,
        driver_numbers=driver_numbers,
        specific_lap=specific_lap,
        save_results=True,
        output_path='../../outputs/integrated_recommendations.csv'
    )
    
    # Display results summary
    if not results.empty:
        print("\nResults summary:")
        print(f"- Total recommendations: {len(results)}")
        print(f"- Drivers analyzed: {results['DriverNumber'].nunique()}")
        print(f"- Recommendation types: {results['action'].unique()}")
        
        # Show a sample of the recommendations
        print("\nSample recommendations:")
        display(results.head())
    
except FileNotFoundError as e:
    print(f"File not found: {str(e)}")
    print("Please adjust the file paths to match your actual file locations")
    
except Exception as e:
    print(f"Error during execution: {str(e)}")

Testing complete strategy pipeline...
Starting complete strategy analysis at 2025-04-18 18:00:59

1. LOADING RACE DATA
----------------------
✅ Loaded lap data: 1180 rows
Data loaded and validated: 1180 rows, 15 columns
Processing Medium tires (ID: 2)...
Processing Hard tires (ID: 3)...
  No laps with new tires for Hard, using TyreAge=2.0 as baseline
Processing Soft tires (ID: 1)...
  No laps with new tires for Soft, using TyreAge=2.0 as baseline
Degradation metrics successfully calculated
Processed data format: 16 features
Created 339 sequences of 5 laps each
Sequences by compound: {1: 158, 2: 147, 3: 34}
Using device: cuda
Global model loaded from: ../../outputs/week5/models/tire_degradation_tcn.pth
Specialized model for compound 1 loaded
Specialized model for compound 2 loaded
Specialized model for compound 3 loaded
Models loaded: 1 global model and 3 specialized models
Prepared tensor for model: shape=torch.Size([339, 5, 16])
Generated ensemble predictions for 339 sequences
Formatt